In [6]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import gcsfs
from google.cloud import bigquery, storage
from google.oauth2 import credentials

In [7]:
def get_blob(blobs):
    for blob in blobs:
        yield blob

In [ ]:
image_dir='/images/'
# plt.figure(figsize=(300,40))

# initialize the GCS client
storage_client = storage.Client()

# get the storage bucket
bucket = storage_client.get_bucket('spectrain')

# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs('spectrain', prefix='Kidney_TX_Data')

i = 0 # counter to use for breaking

# Note: The call returns a response only when the iterator is consumed.
for blob in get_blob(blobs):
    if("output" in blob.name):
        print(i)
        plt.figure(figsize=(300,40), clear=True)
        df=pd.read_csv("gs://spectrain/"+blob.name, sep='\s', header=None)
        df.columns=['x_axis', 'y_axis']
        plt.plot(df['x_axis'], df['y_axis'])
        plt.axis('off')
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        filename = blob.name.split('.')[0] # remove the suffix/file extension
        filename = filename.split('/')[1] # remove the containing directory name from filename
        upload_blob = bucket.blob(image_dir+filename+'_nmr.png')
        upload_blob.upload_from_file(buf, content_type='image/png', rewind=True)
        buf.close()
        plt.close("all")
        plt.close()
        i=i+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
641
642
643
644
645
646
647
648
649
650
651
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
78